<a href="https://colab.research.google.com/github/yseeker/pytorch_templates/blob/main/template_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jul  1 01:15:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install gwpy --quiet

In [ ]:
%%capture
class CFG:
    project_name = 'your project name'
    pretrained_model_name = 'efficientnetv2_rw_s'
    pretrained = True
    prettained_path = '../input/timm_weight/efficientnet_b0_ra-3dd342df.pth'
    mixup_alpha = 0.5
    lr = 1.0e-6
    batch_size= 16
    wandb_note = f'tawara_bs{batch_size}_adamW_default_lr{lr}_alpha{mixup_alpha}'
    input_channels = 3
    out_dim = 1
    colab_or_kaggle = 'colab'
    wandb_exp_name = f'{pretrained_model_name}_{colab_or_kaggle}_{wandb_note}'
    epochs = 60
    num_of_fold = 5
    seed = 42
    patience = 3
    delta = 0.002
    num_workers = 8
    fp16 = True
    checkpoint_path = ''
    patience_mode = 'max'
    patience = 3
    delta = 0.002

In [ ]:
# Kaggle
import os
import json
f = open("/content/drive/My Drive/kaggle/kaggle.json", 'r')
json_data = json.load(f) #JSON形式で読み込む
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']
os.chdir("/content/drive/My Drive/kaggle/working")
os.chdir("/content") #content 直下

In [ ]:
%%capture
!pip install --upgrade albumentations
!pip install --upgrade wandb
!pip install --!pip install colabcode --upgrade
# ColabCode()

import os
import sys
import random
from pathlib import Path
#from tqdm import tqdm
from tqdm.notebook import tqdm
import datetime
import psutil
import gc

import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
import torchvision
from torchsummary import summary

import cv2
from PIL import Image
import albumentations as A
import timm

import wandb
from colabcode import ColabCode
import warnings

warnings.filterwarnings("ignore")

In [ ]:
#If internet is off
tez_path = '../input/tez-lib'
efnet_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
timm_path = '../input/pytorch-image-models-master'
sys.path.append(tez_path)
sys.path.append(efnet_path)
sys.path.append(timm_path)

In [ ]:
# Utils
def set_seed(seed = 0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

def mixup_data(inputs, targets, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = inputs.size()[0]
    index = torch.randperm(batch_size)
    mixed_inputs = lam * inputs + (1 - lam) * inputs[index, :]
    targets_a, targets_b = targets, targets[index]
    
    return mixed_inputs, targets_a, targets_b, lam

def mixup_criterion(criterion, outputs, targets_a, targets_b, lam):
    return lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)

In [ ]:
class AverageMeter:
    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
train_aug = A.Compose(
    [
        A.Resize(p = 1.0, height = 512, width = 512),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5,
                           shift_limit = 0.2, 
                           scale_limit=0.2,
                           rotate_limit=30, 
                           border_mode = cv2.BORDER_REPLICATE),
        A.RandomResizedCrop(
            p = 1.0,
            height = 320,
            width = 320,
            scale = [0.9, 1.0]
        ),
        A.OneOf([
            A.MedianBlur(p=0.3),
            A.MotionBlur(p=0.3)
        ]
        )
    ]
)
df = pd.read_csv('../input/***/train_labels.csv')
df['img_path'] = df['id'].apply(
    lambda x: f'../input/***/train/{x[0]}/{x}.npy'
)
X = df.img_path.values
Y = df.target.values
skf = StratifiedKFold(n_splits = CFG.num_of_fold)

class ClassificationDataset():
    def __init__(self, image_paths, targets, transform = None): 
        self.image_paths = image_paths
        self.targets = targets
        self.transform = None

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item): 
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]

        image_tensor = torch.tensor(image)
        return image_tensor, torch.tensor(targets)

In [ ]:
class BasicNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(CFG.pretrained_model_name, 
                                       pretrained = CFG.pretrained, 
                                       in_chans = CFG.input_channels)
        if not CFG.pretrained: self.model.load_state_dict(torch.load(CFG.pretrained_path))
        self.model.classifier = nn.Linear(self.model.classifier.in_features, CFG.out_dim)
        
    def forward(self, inputs):
        outputs = self.model(inputs)
        return outputs

class Trainer():
    def __init__(
        self,
        model,
        train_dataset,
        valid_dataset = None,
        train_batchsize = 16,
        valid_batchsize = 16,
        valid_targets = None,
        num_workers = 4,
        fp16 = True,
        multiple_GPU = False,
        determinstic = True,
        benchmark = False
    ):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        self.model = model
        self.model.to(self.device)
        self.valid_targets = valid_targets

        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = self.configure_optimizer()
        self.scheduler_after_step = self.configure_scheduler_after_step()
        self.scheduler_after_epoch = self.configure_scheduler_after_epoch()

        torch.backends.cudnn.deterministic = determinstic
        torch.backends.cudnn.benchmark = benchmark
        self.fp16 = fp16
        self.scaler = torch.cuda.amp.GradScaler() 
        
        if num_workers == -1: num_workers = psutil.cpu_count()
        self.multiple_GPU = multiple_GPU
        if multiple_GPU and torch.cuda.device_count() > 1:
            print("Let's use", torch.cuda.device_count(), "GPUs!")
            self = nn.DataParallel(self)
        
        self.train_loader = torch.utils.data.DataLoader(
            dataset = train_dataset, 
            batch_size = train_batchsize,
            shuffle=True, 
            num_workers= num_workers,
            drop_last = True,
            pin_memory = True
        )
        self.valid_loader = torch.utils.data.DataLoader(
            dataset = valid_dataset, 
            batch_size=valid_batchsize,
            shuffle=False, 
            num_workers = num_workers,
            drop_last = False,
            pin_memory = True
        )

    def _init_wandb(self, cfg):
        hyperparams = {
            'batch_size' : cfg.batch_size,
            'epochs' : cfg.epochs
        }
        wandb.init(
            config = hyperparams,
            project= cfg.project_name,
            name=cfg.wandb_exp_name,
        )
        wandb.watch(self.model)

    def configure_optimizer(self):
        opt = torch.optim.Adam(self.model.parameters(), lr=CFG.lr)
        return opt
    
    def configure_scheduler_after_step(self):
        sch = torch.optim.lr_scheduler.OneCycleLR(
            optimizer = self.optimizer,
            epochs = CFG.epochs,
            steps_per_epoch = 2508,
            max_lr = 5.0e-4,
            pct_start = 0.1,
            anneal_strategy = 'cos',
            div_factor = 1.0e+3,
            final_div_factor = 1.0e+3
        )
        return sch

    def configure_scheduler_after_epoch(self):
        return None

    def epoch_metrics(self, outputs, targets):
        return metrics.roc_auc_score(targets, outputs)

    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        if len(np.unique(targets)) > 1: 
            roc_auc = metrics.roc_auc_score(targets, outputs)
        else: roc_auc = 1.0
        return roc_auc

    def train_one_step(self, inputs, targets):
        inputs = inputs.to(self.device, non_blocking=True)
        targets = targets.to(self.device, non_blocking=True)
        inputs, targets_a, targets_b, lam = mixup_data(inputs, 
                                                      targets,
                                                      alpha= CFG.mixup_alpha)
        self.optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            if self.fp16:
                with torch.cuda.amp.autocast(self.fp16):
                    outputs = self.model(inputs)
                    #loss = self.criterion(outputs, targets.view(-1, 1))
                    loss = mixup_criterion(self.criterion,
                                                outputs, 
                                                targets_a.view(-1, 1),
                                                targets_b.view(-1, 1), 
                                                lam)
                    metrics = self.monitor_metrics(outputs, targets)
                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                outputs = self(inputs)
                metrics = self.monitor_metrics(outputs, targets)
                loss = self.criterion(outputs, targets.view(-1, 1))
                loss.backward()
                self.optimizer.step()
            if self.scheduler_after_step:
                self.scheduler_after_step.step()
        return outputs, loss, metrics

    def validate_one_step(self, inputs, targets = None):
        inputs = inputs.to(self.device, non_blocking=True)
        if targets is not None:
            targets = targets.to(self.device, non_blocking=True)
            with torch.no_grad():
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets.view(-1, 1))
                metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        else:
            outputs = self(inputs)
            return outputs, None, None

    def predict_one_step(self, inputs):
        outputs, _, _ = self.validate_one_step(inputs)
        return outputs
        
    def train_one_epoch(self, data_loader):
        self.model.train()
        running_loss, running_metrics = AverageMeter(), AverageMeter()
        tk0 = tqdm(data_loader, total=len(data_loader), position = 0, leave = True)
        for b_idx, (inputs, targets) in enumerate(tk0):
            preds_one_batch, loss, metrics = self.train_one_step(inputs, targets)
            running_loss.update(loss.item(), data_loader.batch_size)
            running_metrics.update(metrics, data_loader.batch_size)
            current_lr = self.optimizer.param_groups[0]['lr'] 
            wandb.log({
                "train/step" : b_idx,
                "train/loss_step": running_loss.avg,
                "lr": current_lr 
                })
            tk0.set_postfix(train_loss=running_loss.avg, train_step_metrics = running_metrics.avg, stage="train", lr = current_lr)
        if self.scheduler_after_epoch:
            self.scheduler_after_epoch.step()
        tk0.close()
        return running_loss.avg

    def validate_one_epoch(self, data_loader):
        self.model.eval()
        running_loss, running_metrics = AverageMeter(), AverageMeter()
        preds_list = []
        tk0 = tqdm(data_loader, total=len(data_loader), position = 0, leave = True)
        for b_idx, (inputs, targets) in enumerate(tk0):
            preds_one_batch, loss, metrics = self.validate_one_step(inputs, targets)
            preds_list.append(preds_one_batch.cpu().detach().numpy())
            running_loss.update(loss.item(), data_loader.batch_size)
            running_metrics.update(metrics, data_loader.batch_size)
            tk0.set_postfix(valid_loss = running_loss.avg, validate_step_metrics = running_metrics.avg, stage="validation")
            wandb.log({
                "valid/step" : b_idx,
                "valid/metric_step" : running_metrics.avg,
                "valid/loss": running_loss.avg, 
                })
        preds_arr = np.concatenate(preds_list)
        valid_metric_val = self.epoch_metrics(preds_arr, self.valid_targets)
        tk0.close()
        return valid_metric_val, running_loss.avg

    def predict(
        self,
        dataset,
        batch_size = 16,
        num_workers = 8,
    ):
        self.model.eval()
        self.test_loader =  torch.utils.data.DataLoader(
            dataset = test_dataset, 
            batch_size = batch_size,
            shuffle = False, 
            num_workers= num_workers,
            drop_last = False,
            pin_memory = True
        )
        preds_list = []
        tk0 = tqdm(data_loader, total=len(self.test_loader), position = 0, leave = True)
        for b_idx, (inputs, targets) in enumerate(tk0):
            preds_one_batch = self.predict_one_step(inputs, targets)
            preds_list.append(preds_one_batch.cpu().detach().numpy())
            tk0.set_postfix(stage="inference")
        tk0.close()
        preds_arr = np.concatenate(preds_list)
        return preds_arr

    def save(self, model_path):
        model_state_dict = self.state_dict()
        if self.optimizer is not None:
            opt_state_dict = self.optimizer.state_dict()
        else:
            opt_state_dict = None
        if self.scheduler_after_step is not None:
            sch_state_dict_after_step = self.scheduler_after_step.state_dict()
        else:
            sch_state_dict_after_step = None
        if self.scheduler_after_epoch is not None:
            sch_state_dict_after_epoch = self.scheduler_after_epoch.state_dict()
        else:
            sch_state_dict_after_epoch = None
        model_dict = {}
        model_dict["state_dict"] = model_state_dict
        model_dict["optimizer"] = opt_state_dict
        model_dict["scheduler_after_step"] = sch_state_dict_after_step
        model_dict["scheduler_after_epoch"] = sch_state_dict_after_epoch
        model_dict["epoch"] = self.current_epoch
        model_dict["fp16"] = self.fp16
        model_dict["multiple_GPU"] = self.multiple_GPU
        torch.save(model_dict, model_path)

    def load(self, model_path):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        if next(self.model.parameters()).device != self.device:
            self.to(self.device)
        model_dict = torch.load(model_path, map_location=torch.device(device))
        self.load_state_dict(model_dict["state_dict"])

    def fit(
        self,
        cfg,
        epochs = 5,
        checkpoint_save_path = '',
        mode = 'max',
        patience = 10,
        delta = 0.001,
    ):
        set_seed(CFG.seed)
        self._init_wandb(cfg)
        path_directory = Path(checkpoint_save_path)
        if mode == 'max':
            current_best_valid_metrics = -float('inf')
        else:
            current_best_valid_metrics= float('inf')
        early_stopping_counter = 0

        for epoch in range(epochs):
            train_loss = self.train_one_epoch(self.train_loader)
            if valid_dataset:
                valid_metrics, valid_loss = self.validate_one_epoch(self.valid_loader)
                # Early Stopping and save at the check points.
                if mode == 'max':
                    if valid_metrics < current_best_valid_metrics + delta:
                        early_stopping_counter += 1
                        print(f'EarlyStopping counter: {early_stopping_counter} out of {patience}')
                        if early_stopping_counter >= patience: break
                    else:
                        print(f"Validation score improved ({current_best_valid_metrics} --> {valid_metrics}). Saving the check point!")
                        current_best_validmetrics= valid_metrics
                        self.save(checkpoint_save_path / f"{cfg.pretrained_model_name}_epoch{epoch}.cpt" )
                else:
                    if valid_metrics > current_best_valid_metrics - delta:
                        early_stopping_counter += 1
                        print(f'EarlyStopping counter: {early_stopping_counter} out of {patience}')
                        if early_stopping_counter >= patience: break
                    else:
                        print(f"Validation score improved ({current_best_valid_metrics} --> {valid_metrics}). Saving the check point!")
                        current_best_valid_metrics = valid_metrics
                        self.save(checkpoint_save_path / f"{cfg.pretrained_model_name}_epoch{epoch}.cpt" )
                        
            #writer.add_scalar("Loss/train", 1.0, epoch)
            print(f'epoch: {epoch}, validate_epoch_metrics : {valid_metrics}')
            wandb.log({
                "epoch" : epoch,
                "train/loss" : train_loss,
                "valid/loss" : valid_loss,
                "valid/metric" : valid_metrics,
                })
        wandb.finish()
        torch.cuda.empty_cache()
        gc.collect()

In [ ]:
for fold_cnt, (train_index, test_index) in enumerate(skf.split(X, Y), 1):
    train_images, valid_images = X[train_index], X[train_index]
    train_targets, valid_targets = Y[train_index], Y[test_index]

    train_dataset = ClassificationDataset(
        image_paths=train_images, 
        targets=train_targets, 
        transform = train_aug
    )
    valid_dataset = ClassificationDataset(
        image_paths=valid_images, 
        targets=valid_targets, 
        transform = None
    )
    model = BasicNN()

    trainer = Trainer(
        model = model,
        train_dataset = train_dataset,
        valid_dataset = valid_dataset,
        valid_targets = valid_targets,
        train_batchsize = CFG.batch_size,
        valid_batchsize = CFG.batch_size,
        num_workers = CFG.num_workers,
        fp16 = CFG.fp16,
    )

    trainer.fit(
        cfg = CFG,
        epochs = CFG.epochs,
        checkpoint_save_path = CFG.checkpoint_path,
        mode = CFG.patience_mode,
        patience = CFG.patience,
        delta = CFG.delta
    )